In [ ]:
from pyspark.sql import SparkSession, types
from pyspark.sql.functions import pandas_udf
import pandas as pd

In [22]:
spark = SparkSession \
          .builder \
          .master('local[*]') \
          .appName('test') \
          .getOrCreate()

## Measurements

In [108]:
measurements_schema = types.StructType([
  types.StructField('day', types.DateType(), nullable=False),
  types.StructField('interval', types.IntegerType(), nullable=False),
  types.StructField('detid', types.StringType(), nullable=False),
  types.StructField('flow', types.IntegerType(), nullable=True),
  types.StructField('occ', types.FloatType(), nullable=True),
  types.StructField('error', types.IntegerType(), nullable=True),
  types.StructField('city', types.StringType(), nullable=False),
  types.StructField('speed', types.FloatType(), nullable=True)
])

In [109]:
measurements_df = spark.read \
                    .option("header", True) \
                    .schema(measurements_schema) \
                    .csv('./data/measurements_test.csv')

In [110]:
measurements_df.show(10)

+----------+--------+--------+----+----+-----+--------+-----+
|       day|interval|   detid|flow| occ|error|    city|speed|
+----------+--------+--------+----+----+-----+--------+-----+
|2017-05-06|       0|06.X-2li|  12| 0.0|    1|augsburg| NULL|
|2017-05-06|     300|06.X-2li|  12| 0.0|    1|augsburg| NULL|
|2017-05-06|     600|06.X-2li|  12| 0.0|    1|augsburg| NULL|
|2017-05-06|     900|06.X-2li|  16| 0.0|    1|augsburg| NULL|
|2017-05-06|    1200|06.X-2li|  16| 0.0|    1|augsburg| NULL|
|2017-05-06|    1500|06.X-2li|  16| 0.0|    1|augsburg| NULL|
|2017-05-06|    1800|06.X-2li|  20| 0.0|    1|augsburg| NULL|
|2017-05-06|    2100|06.X-2li|  20| 0.0|    1|augsburg| NULL|
|2017-05-06|    2400|06.X-2li|  20| 0.0|    1|augsburg| NULL|
|2017-05-06|    2700|06.X-2li|   8|0.01|    1|augsburg| NULL|
+----------+--------+--------+----+----+-----+--------+-----+
only showing top 10 rows



In [111]:
@pandas_udf(types.TimestampType())
def get_timestamp(days: pd.Series, intervals: pd.Series) -> pd.Series:
  dts = pd.to_datetime(days)
  deltas = pd.to_timedelta(intervals, unit='s')
  return dts + deltas

In [ ]:
# @udf(types.TimestampType())
# def get_timestamp(day, interval):
#   dt = pd.to_datetime(day)
#   delta = pd.to_timedelta(interval, unit='S')
#   return dt + delta

In [112]:
measurements_df = measurements_df.withColumn('ts', get_timestamp('day', 'interval'))

In [113]:
measurements_df.show(5)

+----------+--------+--------+----+---+-----+--------+-----+-------------------+
|       day|interval|   detid|flow|occ|error|    city|speed|                 ts|
+----------+--------+--------+----+---+-----+--------+-----+-------------------+
|2017-05-06|       0|06.X-2li|  12|0.0|    1|augsburg| NULL|2017-05-06 00:00:00|
|2017-05-06|     300|06.X-2li|  12|0.0|    1|augsburg| NULL|2017-05-06 00:05:00|
|2017-05-06|     600|06.X-2li|  12|0.0|    1|augsburg| NULL|2017-05-06 00:10:00|
|2017-05-06|     900|06.X-2li|  16|0.0|    1|augsburg| NULL|2017-05-06 00:15:00|
|2017-05-06|    1200|06.X-2li|  16|0.0|    1|augsburg| NULL|2017-05-06 00:20:00|
+----------+--------+--------+----+---+-----+--------+-----+-------------------+
only showing top 5 rows



## Detectors

## Links

In [13]:
spark.stop()